In [30]:
import pandas as pd
import os

def preprocess_mutual_fund_data(input_path: str, output_path: str, n_rows: int = 1000000):
    # Step 1: Load dataset
    df = pd.read_csv(input_path, nrows=n_rows)

    # Step 2: Drop rows with missing NAV or Date
    df.dropna(subset=["NAV", "Date"], inplace=True)

    # Step 3: Convert Date and filter after 2016
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y', errors='coerce')
    df.dropna(subset=["Date"], inplace=True)
    df = df[df['Date'] >= pd.to_datetime("2016-01-01")]

    # Step 4: Sort by Scheme_Code and Date
    df.sort_values(by=["Scheme_Code", "Date"], inplace=True)

    # Step 5: Keep schemes with sufficient data
    valid_schemes = df["Scheme_Code"].value_counts()
    keep_schemes = valid_schemes[valid_schemes >= 100].index
    df = df[df["Scheme_Code"].isin(keep_schemes)]

    # Step 6: Save cleaned dataset
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False)
    print(f"Preprocessed full dataset saved to: {output_path}")
    return df

# --- Main Processing ---

# Run preprocessing
df = preprocess_mutual_fund_data(
    input_path=r"C:/Users/BALA/OneDrive - University of Hertfordshire/Desktop/mutual-fund-recommender/data/raw/Mutual_Funds.csv",
    output_path=r"C:/Users/BALA/OneDrive - University of Hertfordshire/Desktop/mutual-fund-recommender/data/processed/preprocessed_mutual_funds.csv"
)

# Step 7: Group and filter to find top 5 unique Fund_Houses
scheme_counts = df['Scheme_Code'].value_counts()
valid_schemes = scheme_counts[scheme_counts >= 100].index
filtered_df = df[df['Scheme_Code'].isin(valid_schemes)]

# Step 8: Create summary and select top 5 per unique Fund_House
scheme_summary = (
    filtered_df.groupby('Scheme_Code')
    .agg(Fund_House=('Fund_House', 'first'), Row_Count=('NAV', 'count'))
    .reset_index()
)

top_5_schemes = (
    scheme_summary.sort_values(by='Row_Count', ascending=False)
    .drop_duplicates(subset='Fund_House')
    .head(5)
)

# Step 9: Filter full data for selected Scheme_Codes
top_05_unique_df = filtered_df[filtered_df['Scheme_Code'].isin(top_5_schemes['Scheme_Code'])]

# Step 10: Save selected NAV data for modeling
os.makedirs("data/processed", exist_ok=True)
top_05_unique_df[
    ['Fund_House', 'Scheme_Type', 'Scheme_Category', 'Scheme_Code', 'Scheme_Name', 'Date', 'NAV']
].to_csv("C:/Users/BALA/OneDrive - University of Hertfordshire/Desktop/mutual-fund-recommender/data/processed/top5_scheme_data.csv", index=False)

print("Top 5 scheme NAV data saved to: data/processed/top5_scheme_data.csv")


Preprocessed full dataset saved to: C:/Users/BALA/OneDrive - University of Hertfordshire/Desktop/mutual-fund-recommender/data/processed/preprocessed_mutual_funds.csv
Top 5 scheme NAV data saved to: data/processed/top5_scheme_data.csv


In [31]:
summary_df = (
    filtered_df.groupby('Scheme_Code')
    .agg({
        'Fund_House': 'first',
        'Scheme_Type': 'first',
        'Scheme_Category': 'first',
        'Scheme_Name': 'first',
        'NAV': 'count'
    })
    .rename(columns={'NAV': 'Row_Count'})
    .reset_index()
)

# Sort and pick top 5 per unique Fund_House
top_summary = (
    summary_df.sort_values(by='Row_Count', ascending=False)
    .drop_duplicates(subset='Fund_House')
    .head(5)
)

# Save one-row-per-scheme summary
top_summary.to_csv("C:/Users/BALA/OneDrive - University of Hertfordshire/Desktop/mutual-fund-recommender/data/processed/top5_scheme_summary.csv", index=False)
print("Saved: top5_scheme_summary.csv")


Saved: top5_scheme_summary.csv
